# Make graph

In [32]:
file = "src/cond-mat.hg2"

In [33]:
FIN = open(file, "r")

In [34]:
PERIOD1 = 1995
PERIOD2 = 1996
PERIOD3 = 1997
PERIOD4 = 1998
PERIOD5 = 1999
PERIOD6 = 2000

In [35]:
from datetime import tzinfo, timedelta, datetime

ZERO = timedelta(0)
HOUR = timedelta(hours=1)

# A UTC class.

class UTC(tzinfo):
    """UTC"""

    def utcoffset(self, dt):
        return ZERO

    def tzname(self, dt):
        return "UTC"

    def dst(self, dt):
        return ZERO

utc = UTC()

In [36]:
reading_edges = False
import re, datetime

train_set = set()
test_set = set()

def read_edge(line, train_set, test_set):
    m = re.search(r"(\d+) (\d+) \[ (.*) \]", line)
    time = int(m.group(1))
    edgeCount = int(m.group(2))
    edgeList = m.group(3).split()
    is_train = (PERIOD1 <= datetime.datetime.fromtimestamp(time, utc).year <= PERIOD5)
    is_test = (datetime.datetime.fromtimestamp(time, utc).year == PERIOD6)     
    
    for i in range(edgeCount):
        for j in range(i + 1, edgeCount):
            if is_train:
                train_set.add( edgeList[i] + " " + edgeList[j])
                train_set.add( edgeList[j] + " " + edgeList[i])
            if is_test:
                test_set.add( edgeList[i] + " " + edgeList[j])
                test_set.add( edgeList[j] + " " + edgeList[i])

for line in FIN:
    if reading_edges:
        read_edge(line, train_set, test_set)
    if "Hyperedges" in line:
        reading_edges = True

In [37]:
len(train_set), len(test_set)

(80646, 41032)

In [40]:
train_set_undirected = set()
test_set_undirected = set()
def make_undirected(s, s_undirected):
    for line in s:
        u, w = line.split()
        line_reverted = w + ' ' + u
        if line not in s_undirected and line_reverted not in s_undirected:
            s_undirected.add(line)

make_undirected(train_set, train_set_undirected)
make_undirected(test_set, test_set_undirected)

len(train_set_undirected), len(test_set_undirected)

(40325, 20518)

In [38]:
fout_train = open('train.in', 'w')
for line in train_set:
    fout_train.write(line + '\n')
fout_train.close()

fout_test = open('test.out', 'w')
for line in test_set:
    fout_test.write(line + '\n')
fout_test.close()